In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import torch
import torch.nn as nn
import torch.nn.utils.prune as prune
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import os


In [3]:
device = torch.device("cpu")

baseline_model_path = "/content/drive/MyDrive/AI_MODEL_OPTIMIZATION/models/mobilenetv2_cifar10_baseline.pth"
pruned_model_path = "/content/drive/MyDrive/AI_MODEL_OPTIMIZATION/models/mobilenetv2_pruned.pth"


In [4]:
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)


100%|██████████| 170M/170M [00:05<00:00, 28.4MB/s]


In [5]:
model = models.mobilenet_v2(pretrained=False)
model.classifier[1] = nn.Linear(model.last_channel, 10)
model.load_state_dict(torch.load(baseline_model_path, map_location=device))
model.to(device)
model.eval()


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [6]:
def evaluate(model):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = 100 * correct / total
    print(f"Accuracy: {acc:.2f}%")
    return acc


In [7]:
print("Baseline Model:")
baseline_acc = evaluate(model)


Baseline Model:
Accuracy: 58.01%


In [8]:
parameters_to_prune = []
for module_name, module in model.named_modules():
    if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
        parameters_to_prune.append((module, 'weight'))

# Apply unstructured pruning
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.5,  # prune 50% of weights
)

print(" Pruning complete.")


 Pruning complete.


In [9]:
print("Pruned Model:")
pruned_acc = evaluate(model)
print(f" Pruned Accuracy: {pruned_acc:.2f}%")

Pruned Model:
Accuracy: 15.67%
 Pruned Accuracy: 15.67%


In [ ]:
torch.save(model.state_dict(), pruned_model_path)
print(f" Pruned model saved to: {pruned_model_path}")


💾 Pruned model saved to: /content/drive/MyDrive/AI_MODEL_OPTIMIZATION/models/mobilenetv2_pruned.pth


In [11]:
size_baseline = os.path.getsize(baseline_model_path) / 1e6
size_pruned = os.path.getsize(pruned_model_path) / 1e6

print(f"Baseline Model Size: {size_baseline:.2f} MB")
print(f"Pruned Model Size:   {size_pruned:.2f} MB")
print(f"Reduction: {(size_baseline - size_pruned)/size_baseline*100:.2f}%")


Baseline Model Size: 9.19 MB
Pruned Model Size:   18.02 MB
Reduction: -95.98%
